# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2025/11/11/

In [7]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [12]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 8 relevant links


{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'portfolio page', 'url': 'https://edwarddonner.com/curriculum/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook profile',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [9]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [14]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 8 relevant links


{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'resume page', 'url': 'https://edwarddonner.com/curriculum/'},
  {'type': 'blog', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'external company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook page',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [15]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 4 relevant links


{'links': [{'type': 'about page', 'url': 'https://huggingface.co/brand'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'community/forum', 'url': 'https://discuss.huggingface.co'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [16]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [17]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 12 relevant links


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
Lightricks/LTX-2
Updated
5 days ago
•
861k
•
900
fal/Qwen-Image-Edit-2511-Multiple-Angles-LoRA
Updated
6 days ago
•
30.5k
•
550
tencent/HY-MT1.5-1.8B
Updated
13 days ago
•
11.7k
•
744
nvidia/nemotron-speech-streaming-en-0.6b
Updated
8 days ago
•
3.16k
•
338
LiquidAI/LFM2.5-1.2B-Instruct
Updated
5 days ago
•
15.1k
•
310
Browse 2M+ models
Spaces
Running
on
Zero
Featured
614
Qwen Image Multiple Angles 3D Camera
🎥
614
Adjust camera angles in images using 3D controls or sliders
Running
Featured
4.17k
Wan2.2 Animate
👁
4.17k
Wan2.2 Animate
Running
on
Zero
1.38k
Z Image Turbo
🖼
1.38k
Generate stunning images from text descriptions in sec

In [18]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [19]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [20]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 7 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nLightricks/LTX-2\nUpdated\n5 days ago\n•\n861k\n•\n900\nfal/Qwen-Image-Edit-2511-Multiple-Angles-LoRA\nUpdated\n6 days ago\n•\n30.5k\n•\n550\ntencent/HY-MT1.5-1.8B\nUpdated\n13 days ago\n•\n11.7k\n•\n744\nnvidia/nemotron-speech-streaming-en-0.6b\nUpdated\n8 days ago\n•\n3.16k\n•\n338\nLiquidAI/LFM2.5-1.2B-Instruct\nUpdated\n5 days ago\n•\n15.1k\n•\n310\nBrowse 2M+ models\nSpaces\nRunning\no

In [21]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [22]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 3 relevant links


# Hugging Face Brochure  

---

## About Hugging Face  
Hugging Face is a vibrant AI community and a leading collaboration platform dedicated to advancing machine learning (ML). At its core, Hugging Face empowers ML engineers, researchers, and enthusiasts globally to share, explore, and innovate through open-source models, datasets, and applications. The company stands at the forefront of the AI revolution by enabling an open and ethical AI future.  

Its platform serves as a central hub where over 2 million models and 500,000+ datasets are hosted, facilitating rapid development and deployment across modalities including text, image, video, audio, and 3D. Hugging Face’s open source stack accelerates workflows while providing paid compute to further support ML endeavors.  

---

## Company Culture  
- **Community-Driven:** Hugging Face values open collaboration, encouraging contributors to build portfolios and share their ML projects with a worldwide audience.  
- **Inclusivity & Ethics:** The company fosters an ethical approach to AI development, dedicated to transparency and inclusiveness in machine learning innovation.  
- **Innovation-Focused:** With a talented science team exploring the cutting-edge of AI technology, Hugging Face nurtures creativity and breakthroughs in ML research and applications.  
- **Supportive Ecosystem:** The platform supports developers and enterprises with accessible tools, documentation, and an extensive community forum for continuous learning and support.  

---

## Products & Services  
- **Model Hub:** Explore and share over 2 million machine learning models, covering a variety of use cases and domains.  
- **Datasets:** Access a vast repository of over 500,000 datasets for training and evaluation across diverse fields.  
- **Spaces:** Host and demo AI applications directly in the browser with community-powered Spaces. Popular apps include advanced text-to-image generation, animation, and 3D image manipulations.  
- **Compute Solutions:** Paid compute resources to accelerate ML training and inference.  
- **Enterprise Solutions:** Customized offerings to help businesses integrate AI and ML technologies effectively.  

---

## Customers and Community  
Hugging Face serves a diverse user base ranging from individual ML researchers and developers to large enterprises involved in AI innovation. Its open platform fosters a thriving community where contributions come from academia, startups, and major tech players alike. The collaborative environment promotes shared growth and rapid advancement in machine learning technologies.

---

## Career Opportunities  
Hugging Face is continuously seeking passionate individuals ready to contribute to the AI future. Careers span multiple domains including research science, software engineering, product development, and community management. The company thrives on diversity, encourages innovation, and supports career growth through collaboration with world-class AI talent.  

Explore current openings and join a mission-driven workplace focused on shaping the future of AI by visiting their [Careers page](https://huggingface.co/careers).

---

## Connect With Hugging Face  
- **Website:** [huggingface.co](https://huggingface.co)  
- **GitHub:** github.com/huggingface  
- **Twitter:** @huggingface  
- **LinkedIn:** linkedin.com/company/hugging-face  
- **Discord:** Community chat and support  

---

**Hugging Face** – Building the future of AI through open innovation and collaboration.  
Join the community to create, discover, and accelerate machine learning worldwide.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [23]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [24]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links


# Hugging Face Brochure

---

## About Hugging Face  
Hugging Face is the premier collaboration platform for the global machine learning community. It serves as a central hub where engineers, scientists, and AI enthusiasts can **share, explore, and experiment with open-source machine learning models, datasets, and applications**. Hugging Face empowers the next generation of AI creators, fostering an open, ethical, and collaborative AI future.

At the core of Hugging Face is a vibrant, fast-growing community that uses some of the most popular ML libraries and tools. Their expert science team continuously pushes the boundaries of AI technology, making Hugging Face a key player in the AI revolution.

---

## What We Offer  

### The Hugging Face Hub  
- **2M+ Models:** Access a vast collection of machine learning models covering text, image, video, audio, and 3D modalities.  
- **500k+ Datasets:** Comprehensive datasets to fuel your machine learning projects.  
- **Spaces:** Host and explore ML applications and demos, enabling interactive AI experiences.  
- **Community Collaboration:** Tools for sharing and discovering models and datasets, plus building a professional ML portfolio.

### Platform Advantages  
- **Open Source Stack:** Accelerate ML development with our full open source ecosystem.  
- **Multi-Modal Support:** Work seamlessly across text, images, video, audio, and even 3D data.  
- **Collaborative Features:** Host unlimited public models, datasets, and applications to collaborate efficiently.  
- **Accelerated Compute:** Paid compute options to speed up your AI experimentation.

---

## Enterprise Solutions  

Hugging Face offers a robust enterprise platform designed to scale AI development securely across organizations. Key features include:

- **Enterprise-Grade Security:** Single Sign-On (SSO), audit logs, and granular access controls to manage your data confidently.  
- **Private Storage & Dataset Viewer:** Additional private storage with tools to collaboratively view datasets internally.  
- **Advanced Compute:** Scalable options like ZeroGPU with quota boosts to handle demanding workloads.  
- **Analytics Dashboard:** Track repository usage and monitor team activity in real-time.  
- **Flexible Plans:** Team subscriptions start at $20/user/month, with custom enterprise contracts available.

---

## Company Culture  

Hugging Face fosters a culture of **openness, collaboration, and innovation** centered on building a better, more ethical AI future together. The company believes in sharing knowledge freely, supporting a thriving global community of machine learning practitioners, and pushing technological boundaries with a strong ethical compass.

---

## Careers Opportunities  

Join Hugging Face to be part of the AI revolution! The company seeks talented engineers, scientists, and AI enthusiasts passionate about:

- Advancing open-source machine learning technology  
- Building collaborative tools for the AI community  
- Exploring state-of-the-art AI research and production  
- Contributing to an ethical and inclusive AI ecosystem

Working at Hugging Face means being part of a mission-driven company at the forefront of AI innovation, with opportunities to impact the future of technology on a global scale.

---

## Connect With Us  

- **Website:** https://huggingface.co  
- **Community:** Join discussions on GitHub, Discord, Twitter & LinkedIn  
- **Documentation & Learning:** Extensive docs and tutorials to help you get started quickly  
- **Blog & Forum:** Stay updated with the latest AI trends, product updates, and community stories

---

**Hugging Face** — The AI community building the future of machine learning.  
Create. Collaborate. Accelerate. Join us today!

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

In [25]:
stream_brochure("Digilogue Technologies", "https://www.digilogue.ca")

Selecting relevant links for https://www.digilogue.ca by calling gpt-5-nano
Found 1 relevant links


# Digilogue Technologies Ltd.

**Digitizing an analogue world one bit at a time.**

---

## About Us

Founded and led by seasoned software consultant Gary Black, Digilogue Technologies Ltd. is a Toronto-based software consulting and innovation company. With over 20 years of expertise in engineering, high-performance computing, and enterprise architecture, we specialize in delivering cutting-edge software engineering solutions and enterprise consulting.

Our mission is to help businesses **scale**, **optimize**, and **innovate** by leveraging deep technical insight and precision-crafted solutions. Our core focus areas include microservices, distributed systems, AI-driven architecture, and cloud-native development.

---

## What We Do

Digilogue Technologies offers a comprehensive range of software consulting services tailored to meet complex business and technological challenges:

- **Software Development & Tooling**
- **Infrastructure & Configuration Management**
- **Software Testing & Documentation**
- **Software Development Methodologies**
- **Architecture & Design**
- **Project Discovery & Innovation Development Projects (IDP)**
- **Cloud Solutions**
- **DevOps Practices**
- **Artificial Intelligence Integration**

---

## Industry Expertise

We bring specialized knowledge to key sectors such as:

- **Finance & Commercial Banking:** Payments, loans, deposits, credit cards, and financial technologies operations.
- **Trading & Investment Platforms:** Handling multi-asset class development and platform engineering.
- **Public Sector:** Backend systems integration including police national computer databases.
- **Enterprise Web Applications:** CRM systems focused on print, recruitment, and business networking.
- **Manufacturing Services:** Electronic manufacturing services with in-circuit developments.

---

## Our Approach & Culture

At Digilogue Technologies, we value **innovation**, **precision**, and **collaboration**. Every project is a partnership, blending our expertise with client vision to deliver scalable and sustainable technology solutions.

Our culture is founded on:

- Deep technical expertise paired with practical business acumen.
- Agile and transparent development methodologies ensuring quality and responsiveness.
- Commitment to continuous learning, embracing emerging technologies like AI and cloud computing.
- Fostering a collaborative and inclusive environment where ideas thrive.

---

## For Customers

Whether you are a startup looking to build resilient cloud-native applications or an enterprise aiming to modernize legacy systems with AI and microservices, Digilogue tailors solutions that meet your unique needs. Our hands-on consulting and engineering services help you stay ahead in the digital transformation journey.

---

## Career Opportunities

We welcome passionate software engineers, architects, and technology consultants who are eager to work on challenging projects across diverse domains. At Digilogue Technologies, you'll:

- Work with advanced technologies including AI, microservices, and cloud ecosystems.
- Collaborate directly with industry leaders and highly experienced consultants.
- Grow your expertise in a company that values innovation and quality.

If you are driven by problem-solving and innovation, consider joining our team to shape the future of software engineering and enterprise consulting.

---

## Contact

**Location:** Toronto, Ontario, Canada  
**Website:** [Digilogue Technologies Ltd.](#)  
**Get in touch to discuss your project or career opportunities.**

---

**Digilogue Technologies Ltd.**  
Innovative Software Engineering Solutions & Enterprise Consulting  
_Digitizing an analogue world one bit at a time._

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>